### Use case notebook for the CCB cadence python package ###

In [53]:
import os
import glob
import numpy as np
import pandas as pd
from pathlib import Path
import time
import timeboard as tb
import timeboard.calendars.US as US
import datetime
import holidays
from faker import Faker

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import the package
%load_ext autoreload
%autoreload 2
import cadence
from cadence.mscheduler import Meetings
print(f'Package version: {cadence.__version__}')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Package version: 0.0.post1.dev10+g7c41242.d20240820


### Meeting participants and settings ###

In [8]:
# Local data
data_dir = os.path.join(os.environ.get('HOME'), 'data')
group_members_file_name = 'ccb_names_list.csv'
ccb_names_file = os.path.join(data_dir, group_members_file_name)
tm_raw = pd.read_csv(ccb_names_file)
display(tm_raw)

,name,email,group
0,Andreas,andreas_werdich@hms.harvard.edu,ai-data
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data
2,Nathan,nathan_palmer@hms.harvard.edu,ai-data
3,Sam,samantha_pullman@hms.harvard.edu,ai-data
4,Sreya,sreya_banerjee@hms.harvard.edu,ai-data
5,Andrew,andrew_ghazi@hms.harvard.edu,comp-bio
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio
7,Grey,grey_kuling@hms.harvard.edu,comp-bio
8,Ludwig,ludwig_geistlinger@hms.harvard.edu,comp-bio
9,Tram,tram_nguyen@hms.harvard.edu,comp-bio


### Create a sequence of weekly meetings ###

In [9]:
email_col = 'email'
name_col = 'name'
group_col = 'group'

# Lets do some cleanup of the spreadsheet
# small letters for the column names
tm = tm_raw.copy()
tm.columns = [col.lower() for col in tm.columns]

# small letters for emails and group, first letter capitalize for names
tm[email_col] = tm[email_col].str.lower()
tm[group_col] = tm[group_col].str.lower()
tm[name_col] = tm[name_col].str.title()

# Sort by group and then name
tm = tm.sort_values(by=['group', 'name'], ascending=True).reset_index(drop=True)

# Mark names that are not presenting
rm_name = ['nathan', 'ludwig']
rm_name = [nm.title() for nm in rm_name]
tm = tm.assign(presenting=True)
tm.loc[tm['name'].isin(rm_name), 'presenting'] = False

# save the name ist as a csv_file
presenter_list_file_name = f'{os.path.splitext(group_members_file_name)[0]}_list.csv'
presenter_list_file = os.path.join(data_dir, presenter_list_file_name)
tm.to_csv(presenter_list_file, index=False)
display(tm)

,name,email,group,presenting
0,Andreas,andreas_werdich@hms.harvard.edu,ai-data,True
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data,True
2,Nathan,nathan_palmer@hms.harvard.edu,ai-data,False
3,Sam,samantha_pullman@hms.harvard.edu,ai-data,True
4,Sreya,sreya_banerjee@hms.harvard.edu,ai-data,True
5,Andrew,andrew_ghazi@hms.harvard.edu,comp-bio,True
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio,True
7,Grey,grey_kuling@hms.harvard.edu,comp-bio,True
8,Ludwig,ludwig_geistlinger@hms.harvard.edu,comp-bio,False
9,Tram,tram_nguyen@hms.harvard.edu,comp-bio,True


In [11]:
# Remove the names that we do not want in the schedule
presenters_df = tm.loc[tm['presenting']==True]
display(presenters_df)

,name,email,group,presenting
0,Andreas,andreas_werdich@hms.harvard.edu,ai-data,True
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data,True
3,Sam,samantha_pullman@hms.harvard.edu,ai-data,True
4,Sreya,sreya_banerjee@hms.harvard.edu,ai-data,True
5,Andrew,andrew_ghazi@hms.harvard.edu,comp-bio,True
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio,True
7,Grey,grey_kuling@hms.harvard.edu,comp-bio,True
9,Tram,tram_nguyen@hms.harvard.edu,comp-bio,True
10,Tyrone,tyrone_lee@hms.harvard.edu,comp-bio,True


In [58]:
# Presenter list
from cadence.mscheduler import Meetings, cyclic_permutate
meet = Meetings(name_df=presenters_df)

presenter_list = meet.presenter_list(random_state=984)
start_name = 'Gerald'
presenter_list = cyclic_permutate(lst_in=presenter_list, name=start_name)
print(presenter_list)

# Switch names
p = presenter_list.copy()
p[6], p[7] = p[7], p[6]
p[7], p[8] = p[8], p[7]

meet.name_list=p
print(meet.name_list)

['Gerald', 'Grey', 'Sam', 'Tram', 'Sreya', 'Tyrone', 'Andrew', 'Andreas', 'Anthony']
['Gerald', 'Grey', 'Sam', 'Tram', 'Sreya', 'Tyrone', 'Andreas', 'Anthony', 'Andrew']


In [64]:
start_date = '2024-08-28'
end_date = '2024-12-31'

cal = meet.create_timeboard(start_date=start_date, 
                            end_date=end_date,
                            start_name='Gerald')

skip_date = '2024-08-28'
skip_name = 'Everyone'
skip_comment = 'Repos, best practices discussion'

cal = meet.skip_date(cal_df=cal, 
                     skip_date=skip_date, 
                     skip_comment=skip_comment,
                     skip_name=skip_name)

display(cal)

/home/andreas/gitrepos/cadence/.venv/lib/python3.11/site-packages/timeboard/when.py:37: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  start_times = pi.to_timestamp(how='start', freq='S')
/home/andreas/gitrepos/cadence/.venv/lib/python3.11/site-packages/timeboard/when.py:37: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  start_times = pi.to_timestamp(how='start', freq='S')


,date,name,email,group,presenting,holiday,comment
0,2024-08-28,Everyone,gerald_kiwanuka@hms.harvard.edu,ai-data,True,False,"Repos, best practices discussion"
1,2024-09-04,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data,True,False,None
2,2024-09-11,Grey,grey_kuling@hms.harvard.edu,comp-bio,True,False,None
3,2024-09-18,Sam,samantha_pullman@hms.harvard.edu,ai-data,True,False,None
4,2024-09-25,Tram,tram_nguyen@hms.harvard.edu,comp-bio,True,False,None
5,2024-10-02,Sreya,sreya_banerjee@hms.harvard.edu,ai-data,True,False,None
6,2024-10-09,Tyrone,tyrone_lee@hms.harvard.edu,comp-bio,True,False,None
7,2024-10-16,Andreas,andreas_werdich@hms.harvard.edu,ai-data,True,False,None
8,2024-10-23,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio,True,False,None
9,2024-10-30,Andrew,andrew_ghazi@hms.harvard.edu,comp-bio,True,False,None


In [66]:
# Save the presentation schedule
save_cols = ['date', 'name', 'comment']
schedule_name = f'ccb_presentations_2024.csv'
schedule_file = os.path.join(data_dir, schedule_name)
cal[save_cols].to_csv(schedule_file, index=False)
display(cal[save_cols])

,date,name,comment
0,2024-08-28,Everyone,"Repos, best practices discussion"
1,2024-09-04,Gerald,None
2,2024-09-11,Grey,None
3,2024-09-18,Sam,None
4,2024-09-25,Tram,None
5,2024-10-02,Sreya,None
6,2024-10-09,Tyrone,None
7,2024-10-16,Andreas,None
8,2024-10-23,Anthony,None
9,2024-10-30,Andrew,None
